### 2025 Attendance Solver (Faculties)

- This is Jypyter version of the attendance issue
- This will serve as solver for Faculties

In [3]:
import pandas as pd
import numpy as np
import math
import warnings

In [29]:
%run util_attendance.py

#### 1. Reading all the files

In [20]:
df_gimt = pd.read_excel('./data/GIMT_MonthlyAttendanceSummaryReport (27).xlsx')
df_gips = pd.read_excel('./data/GIPS_MonthlyAttendanceSummaryReport(43).xlsx')
df_leave_erp = pd.read_csv('./data/Leave report .csv',skiprows=6, encoding='windows-1252')
df_emp = pd.read_csv('./data/GCU080057_Faculty_List.csv', skiprows=6, encoding='windows-1252')

In [21]:
no_working_days = 22

In [30]:
df_gimt_all, df_gimt_in, df_gimt_out = split_file(df_gimt)
df_gimt_merged = merge_files_aug(df_gimt_in, df_gimt_out, no_working_days)

df_gips_all, df_gips_in, df_gips_out = split_file(df_gips)
df_gips_merged = merge_files_aug(df_gips_in, df_gips_out, no_working_days)

In [31]:
print(f"the shape of GIMT - {df_gimt_all.shape}")
print(f"the shape of GIPS - {df_gips_all.shape}")

the shape of GIMT - (118, 56)
the shape of GIPS - (40, 49)


In [32]:
df_fac_all = pd.concat([df_gimt_all, df_gips_all], ignore_index=True)
df_fac_all.fillna(0, inplace=True)

#### 2. Detailed Biometric Attendance (Save 1)

In [33]:
df_fac_all.head(3)

,Emp ID,Names,clock_in_7_26,clock_in_7_27,clock_in_7_28,clock_in_7_29,clock_in_7_30,clock_in_7_31,clock_in_8_01,clock_in_8_02,...,clock_out_8_12,clock_out_8_13,clock_out_8_14,clock_out_8_18,clock_out_8_19,clock_out_8_20,clock_out_8_21,clock_out_8_22,clock_out_8_23,clock_out_8_25
0,GCU010013,Ms. Raisa,11:42:00,0,08:40:00,08:42:00,08:40:00,08:40:00,0,0,...,16:05:00,0,16:03:00,16:20:00,16:21:00,16:20:00,16:20:00,16:20:00,16:20:00,0
1,GCU010017,Ms. Ananya,08:24:00,0,08:33:00,08:24:00,08:24:00,08:27:00,08:24:00,0,...,16:04:00,16:53:00,16:03:00,16:20:00,16:21:00,16:22:00,16:20:00,16:20:00,16:20:00,0
2,GCU020001,Dr. Lipi,08:39:00,0,08:42:00,08:39:00,08:42:00,08:37:00,08:38:00,0,...,16:00:00,16:04:00,15:56:00,16:21:00,16:21:00,16:24:00,16:24:00,16:23:00,16:24:00,0


In [34]:
working_days = calculate_working_days(df_gimt_in)

In [ ]:
#### 3. Observed leaves (Save 2)

In [35]:
df_fac_merged = pd.concat([df_gimt_merged, df_gips_merged], ignore_index=True)
col_index_map = {'Working Days':2,'Present':3,'Absent':4}
df_fac_merged = move_columns(df_fac_merged, col_index_map)
df_fac_merged.head()

,Emp ID,Names,Working Days,Present,Absent,late_flags,early_flags,half_day_flags,AM_abs,PM_abs,days_abs,No_of_late
0,GCU010013,Ms. Raisa,22,18.5,3.5,"[7_26, 8_14]",[],"[7_26, 7_30, 8_14]","[7_26, 8_14]",[7_30],"[8_13, 8_01]",2
1,GCU010017,Ms. Ananya,22,22.0,0.0,[],[],[],[],[],[],0
2,GCU020001,Dr. Lipi,22,21.0,1.0,"[8_04, 8_07, 8_08, 8_11, 8_13, 8_14, 8_18]",[],[],[],[],[8_09],7
3,GCU020002,Dr. Swarnali,22,21.5,0.5,[7_26],[],[7_26],[7_26],[],[],1
4,GCU020003,Dr. Kajal,22,22.0,0.0,[],[],[],[],[],[],0


In [36]:
df_fac_merged.rename(columns={'AM_abs':'actual_AM_abs','PM_abs':'actual_PM_abs','days_abs':'actual_days_abs','No_of_late':'actual_No_of_late'}, inplace=True)
df_fac_merged.head()

,Emp ID,Names,Working Days,Present,Absent,late_flags,early_flags,half_day_flags,actual_AM_abs,actual_PM_abs,actual_days_abs,actual_No_of_late
0,GCU010013,Ms. Raisa,22,18.5,3.5,"[7_26, 8_14]",[],"[7_26, 7_30, 8_14]","[7_26, 8_14]",[7_30],"[8_13, 8_01]",2
1,GCU010017,Ms. Ananya,22,22.0,0.0,[],[],[],[],[],[],0
2,GCU020001,Dr. Lipi,22,21.0,1.0,"[8_04, 8_07, 8_08, 8_11, 8_13, 8_14, 8_18]",[],[],[],[],[8_09],7
3,GCU020002,Dr. Swarnali,22,21.5,0.5,[7_26],[],[7_26],[7_26],[],[],1
4,GCU020003,Dr. Kajal,22,22.0,0.0,[],[],[],[],[],[],0


#### 4. Handling EMP from ERP

In [37]:
df_emp.head(3)

,Serial No,Employee ID,Name,Department,Group Name,User Type,Employee Type,Designation,Joining Date,Email ID,Mobile No.,Date of Birth,Sub Department,Academy
0,1,SYSADMIN,System Administrator,Administration,Admin,-,-,-,-,sambit.kundu@serosoft.in,982365654155,16/08/1990,-,Girijananda Chowdhury University
1,2,APPLICANT_PORTAL_USER,APP_FIRST_NAME APP_MIDDLE_NAME APP_LAST_NAME,-,Admin,-,-,-,-,-,-,-,-,Girijananda Chowdhury University
2,3,mis,MIS Admin,Administration,Admin,-,-,-,-,mayank.pandya@serosoft.in,+91-1111111111,18/03/1999,-,Girijananda Chowdhury University


In [38]:
df_emp = preprocess_emp(df_emp)
df_emp.head()

C:\Users\GUEST123\projects\gcuapps\util_attendance.py:929: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'Employee ID':'Emp ID'}, inplace=True)


,Emp ID,Name,Designation,Department
0,EMP5,Birshu Sarma,-,School Of Engineering and Technology
1,GCU020041,Bhabajit Baruah,Assistant Professor,School Of Engineering and Technology
2,GCU020132,Ian Stewart Suchiang,Assistant Professor,School of Pharmaceutical Science - Guwahati
3,GCU020043,Abhinandan Kalita,Assistant Professor,School Of Engineering and Technology
4,GCU020115,Sunayan Bardoloi,Sr. Professor & HoD,School of Natural Science


In [39]:
df_fac_merged.drop(columns ='Names', axis=1, inplace=True)
df_fac_with_ID = pd.merge(df_fac_merged, df_emp, how='left', on='Emp ID')
col_index_map = {'Name':1,'Designation':2,'Department':3,'Working Days':4,'Present':5,'Absent':6}
df_fac_with_ID = move_columns(df_fac_with_ID, col_index_map)
df_fac_with_ID.head(3)

,Emp ID,Name,Designation,Department,Working Days,Present,Absent,late_flags,early_flags,half_day_flags,actual_AM_abs,actual_PM_abs,actual_days_abs,actual_No_of_late
0,GCU010013,Raisa Choudhury,-,Administration,22,18.5,3.5,"[7_26, 8_14]",[],"[7_26, 7_30, 8_14]","[7_26, 8_14]",[7_30],"[8_13, 8_01]",2
1,GCU010017,Ananya Borah,-,Administration,22,22.0,0.0,[],[],[],[],[],[],0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,22,21.0,1.0,"[8_04, 8_07, 8_08, 8_11, 8_13, 8_14, 8_18]",[],[],[],[],[8_09],7


In [44]:
df_fac_with_ID.columns

Index(['Emp ID', 'Name', 'Designation', 'Department', 'Working Days',
       'Present', 'Absent', 'late_flags', 'early_flags', 'half_day_flags',
       'actual_AM_abs', 'actual_PM_abs', 'actual_days_abs',
       'actual_No_of_late'],
      dtype='object')

In [59]:
df_fac_actual = df_fac_with_ID.copy()

# handling half days
df_fac_actual['actual_half_day'] = df_fac_actual.apply(lambda x: len(x['actual_AM_abs'])+len(x['actual_PM_abs']),axis=1)

# handling full days
df_fac_actual['actual_full_day'] = df_fac_actual.apply(lambda x: len(x['actual_days_abs']),axis=1)
df_fac_actual = df_fac_actual[['Emp ID', 'Name', 'Designation', 'Department', 'Working Days', 'Present', 'Absent', 'actual_half_day', 'actual_full_day', 'actual_No_of_late']]

In [53]:
df_fac_actual.head()

,Emp ID,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late
0,GCU010013,Raisa Choudhury,-,Administration,22,18.5,3.5,3,2,2
1,GCU010017,Ananya Borah,-,Administration,22,22.0,0.0,0,0,0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,22,21.0,1.0,0,1,7
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,22,21.5,0.5,1,0,1
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,22,22.0,0.0,0,0,0


#### 5. Working with Exempted leave

In [60]:
df_fac_exempted = read_and_merge_exempted_leaves('./data/template_exempted.xlsx') #,working_days)
df_fac_exempted.head()

,Emp ID,Name,late_count,half_day_count,full_day_count
0,GCU020001,Dr. Lipi Goswami,7.0,0.0,0.0
1,GCU020002,Dr. Swarnali Pathak,0.0,0.0,0.0
2,GCU020003,Dr. Kajal Dutta,0.0,0.0,0.0
3,GCU020004,Dr. Dipankar Saha,0.0,0.0,0.0
4,GCU020006,Dr. Anindita Bora,3.0,0.0,0.0


In [61]:
df_fac_exempted.rename(columns = {'late_count':'exempt_late','half_day_count':'exempt_half_day','full_day_count':'exempt_full_day'}, inplace=True)
df_fac_exempted.head()

,Emp ID,Name,exempt_late,exempt_half_day,exempt_full_day
0,GCU020001,Dr. Lipi Goswami,7.0,0.0,0.0
1,GCU020002,Dr. Swarnali Pathak,0.0,0.0,0.0
2,GCU020003,Dr. Kajal Dutta,0.0,0.0,0.0
3,GCU020004,Dr. Dipankar Saha,0.0,0.0,0.0
4,GCU020006,Dr. Anindita Bora,3.0,0.0,0.0


#### The final attendance - This merges Bio data and exempted data

In [64]:
#df_fac_exempted.drop('Name', axis=1, inplace=True)
#df_fac_exempted = df_fac_exempted.drop(columns=['Name'])
df_fac_attend = pd.merge(df_fac_actual,df_fac_exempted , how='left', on=['Emp ID'])
df_fac_attend.fillna(0, inplace=True)
df_fac_attend.head()

,Emp ID,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late,exempt_late,exempt_half_day,exempt_full_day
0,GCU010013,Raisa Choudhury,-,Administration,22,18.5,3.5,3,2,2,2.0,0.0,0.0
1,GCU010017,Ananya Borah,-,Administration,22,22.0,0.0,0,0,0,0.0,0.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,22,21.0,1.0,0,1,7,7.0,0.0,0.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,22,21.5,0.5,1,0,1,0.0,0.0,0.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,22,22.0,0.0,0,0,0,0.0,0.0,0.0


In [65]:
df_fac_attend['Half Days'] = df_fac_attend['actual_half_day'] - df_fac_attend['exempt_half_day']
df_fac_attend['Full Days'] = df_fac_attend['actual_full_day'] - df_fac_attend['exempt_full_day']
df_fac_attend['Late'] = df_fac_attend['actual_No_of_late'] - df_fac_attend['exempt_late']
df_fac_attend = df_fac_attend[['Emp ID', 'Name', 'Designation', 'Department', 'Working Days', 'Present', 'Absent','Half Days','Full Days','Late']]
df_fac_attend.head()

,Emp ID,Name,Designation,Department,Working Days,Present,Absent,Half Days,Full Days,Late
0,GCU010013,Raisa Choudhury,-,Administration,22,18.5,3.5,3.0,2.0,0.0
1,GCU010017,Ananya Borah,-,Administration,22,22.0,0.0,0.0,0.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,22,21.0,1.0,0.0,1.0,0.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,22,21.5,0.5,1.0,0.0,1.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,22,22.0,0.0,0.0,0.0,0.0


In [ ]:
#df_fac_attend['Corrected Absent'] = df_fac_attend['Absent']-df_fac_attend['Exempted Leaves']
#df_fac_attend['Present'] = df_fac_attend['Present']+df_fac_attend['Exempted Leaves']
#df_fac_attend.head()

In [66]:
df_fac_attend.shape

(158, 10)

#### 6. Working with Leave from ERP

In [80]:
df_leave_erp.head() 

,Serial No.,Employee ID,Name,Location,Leave Type,Request Date,From Date,To Date,Status,Timeline,Total Days
0,1,GCU020130,Ankur Jyoti Kashyap,Girijananda Chowdhury University-Assam,Duty Leave,18/06/2025,31/07/2025,04/08/2025,Approved,View,5
1,2,GCU020107,Salema Khatun,Girijananda Chowdhury University-Assam,Maternity Leave,20/06/2025,02/07/2025,28/12/2025,Approved,View,180
2,3,GCU030031,Tridib Roy,Girijananda Chowdhury University-Assam,Earned Leave,04/07/2025,21/07/2025,26/07/2025,Approved,View,6
3,4,GCU030126,Rita Das,Girijananda Chowdhury University-Assam,Earned Leave,04/07/2025,20/07/2025,27/07/2025,Approved,View,8
4,5,GCU030011,Nitul Das,Girijananda Chowdhury University-Assam,Earned Leave,07/07/2025,21/07/2025,26/07/2025,Approved,View,6


In [81]:
# corrected leaves
df_leave_erp_summary = calculate_leave_summary_with_wd_leaves(df_leave_erp, working_days) 
#fac_with_EOL = find_corrected_EOL(df_leave_erp, working_days)
#df_leave_erp_summary = pd.merge(df_leave_erp_summary,fac_with_EOL , how='left', on=['Emp ID'])
df_leave_erp_summary.fillna(0, inplace=True)
df_leave_erp_summary.head()

,Emp ID,Name,Duty Leave,Casual Leave,Maternity Leave,Earned Leave,Paternity Leave,Sick Leave,Vacation Leave,Total WD leaves
0,GCU020130,Ankur Jyoti Kashyap,5.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
1,GCU020107,Salema Khatun,0.0,0.0,180.0,0.0,0.0,0.0,0.0,22.0
2,GCU030031,Tridib Roy,0.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0
3,GCU030126,Rita Das,0.0,1.0,0.0,8.0,0.0,0.0,0.0,1.0
4,GCU030011,Nitul Das,0.0,1.0,0.0,6.0,0.0,0.0,0.0,1.0


In [82]:
df_leave_erp_summary['Total WD leaves'] = df_leave_erp_summary['Total WD leaves'] + df_leave_erp_summary['Casual Leave']
cols_to_drop = ['Casual Leave', 'Sick Leave', 'Duty Leave', 'Vacation Leave','Maternity Leave', 'Earned Leave','Paternity Leave']

df_leave_erp_summary.drop(columns = cols_to_drop, axis=1, inplace=True)
df_leave_erp_summary.head()

,Emp ID,Name,Total WD leaves
0,GCU020130,Ankur Jyoti Kashyap,4.0
1,GCU020107,Salema Khatun,22.0
2,GCU030031,Tridib Roy,1.0
3,GCU030126,Rita Das,2.0
4,GCU030011,Nitul Das,2.0


#### 7. Combining Attendance and Leave from ERP

In [84]:
df_leave_erp_summary.drop(columns='Name', axis=1,inplace=True)
df_final_report = pd.merge(df_fac_attend, df_leave_erp_summary , how='left', on=['Emp ID'])
df_final_report.fillna(0, inplace=True)
df_final_report.head()

,Emp ID,Name,Designation,Department,Working Days,Present,Absent,Half Days,Full Days,Late,Total WD leaves
0,GCU010013,Raisa Choudhury,-,Administration,22,18.5,3.5,3.0,2.0,0.0,3.0
1,GCU010017,Ananya Borah,-,Administration,22,22.0,0.0,0.0,0.0,0.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,22,21.0,1.0,0.0,1.0,0.0,1.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,22,21.5,0.5,1.0,0.0,1.0,0.5
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,22,22.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
i need to check from here

In [85]:
#cols_to_delete = ['actual_haf_day','actual_AM_abs','actual_PM_abs','exempt_half_day','exempt_full_day','actual_days_abs','actual_No_of_late','exempt_late']
col_to_sum = ['Half Days','Full Days']
df_final_report = weighted_sum_and_replace_columns(df_final_report, col_to_sum, 'Approved Leaves', [0.5,1.0])
#df_final_report.drop(columns = cols_to_delete, inplace=True)
df_final_report.head()

,Emp ID,Name,Designation,Department,Working Days,Present,Absent,Late,Total WD leaves,Approved Leaves
0,GCU010013,Raisa Choudhury,-,Administration,22,18.5,3.5,0.0,3.0,3.5
1,GCU010017,Ananya Borah,-,Administration,22,22.0,0.0,0.0,0.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,22,21.0,1.0,0.0,1.0,1.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,22,21.5,0.5,1.0,0.5,0.5
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,22,22.0,0.0,0.0,0.0,0.0


In [86]:
df_final_report.sample(10)

,Emp ID,Name,Designation,Department,Working Days,Present,Absent,Late,Total WD leaves,Approved Leaves
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,22,22.0,0.0,0.0,0.0,0.0
113,GCU030101,Bhogeswar Saikia,Workshop Instructor,Admin Non Teaching,22,22.0,0.0,0.0,0.0,0.0
123,GCU020059,Jun Moni Kalita,Associate Professor & HOD,School of Pharmaceutical Science - Guwahati,22,20.5,1.5,3.0,2.5,2.5
132,GCU020074,Bhrigu Kumar Das,Assistant Professor,School of Pharmaceutical Science - Guwahati,22,21.0,1.0,0.0,2.0,2.0
127,GCU020065,Susankar Kushari,Assistant Professor,School of Pharmaceutical Science - Guwahati,22,22.0,0.0,1.0,1.0,1.0
118,GCU020004,Dipankar Saha,Professor & HOD,School of Pharmaceutical Science - Guwahati,22,22.5,-0.5,0.0,0.0,0.5
45,GCU020061,Snehal Kaushik,Associate Professor,School Of Engineering and Technology,22,15.0,7.0,0.0,7.0,7.0
122,GCU020054,Damiki Laloo,Professor & HOD,School of Pharmaceutical Science - Guwahati,22,22.0,0.0,3.0,0.5,1.0
136,GCU020078,Lima Patowary,Assistant Professor,School of Pharmaceutical Science - Guwahati,22,22.5,-0.5,2.0,0.0,0.5
110,GCU030062,Joydeep Gupta,Lab Instructor,School Of Engineering and Technology,22,22.0,0.0,9.0,0.0,0.0


In [75]:
df_final_report['Final EOL'] = df_final_report.apply(lambda x: x['Corrected EOL'] if x['Corrected EOL'] > x['Corrected Absent'] else
    x['Corrected EOL'] + x['Corrected Absent'] - (x['Total WD leaves'] + x['Corrected EOL']),axis=1)
df_final_report.head()

KeyError: 'Corrected Absent'